<a href="https://colab.research.google.com/github/Russel-hunho/DeepLearning/blob/main/pytorch_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Colab에서 Pytorch 사용하기
* Tensorflow와 달리, Pytorch는 Colab의 기본 설치 라이브러리가 아니다..!
> 처음 시작시 하기 명령을 통해 설치해줘야 함
>> !pip3 install torch
>> !pip3 install torchvision

* 무료버전의 경우, 12시간마다 런타임이 재설정 된다
> 12시간마다 GPU 설정을 해줘야 함
>> 상단[런타임]-[런타임유형변경]- [하드웨어가속기]: GPU

#1. 미니 배치와 데이터 로드
https://wikidocs.net/55580

-. 데이터 로드법

-. 미니 배치 경사 하강법(Minibatch Gradient Descent)

###1.1. 미니 배치

미니 배치(Mini Batch): 전체 Training Data를 작은 단위로 나누어서 학습하는 개념

> 기본적으로, 학습 Input Data는 Data 여러개를 묶어 행렬로 만들어, Batch 단위로 학습에 사용된다. (= 배치 경사 하강법)

> Batch의 크기가 크면 클수록, 가중치 값이 최적값에 수렴하는 과정이 매우 안정적이지만, 계산량이 너무 많이 듬

> 미니 배치로 나누어 진행하면 훈련 속도가 빨라진다!

***Iteration, Epoch, Batch Size***

Total data가 2000개일 때,
이를 Mini Batch 10개로 나누면,
> Iteration = 10
> Batch Size = 200

**Epoch** = 총 학습 수 = 모든 data가 학습에 사용된 횟수

<->

**Iteration** = 한 Epoch에서 진행되는 학습의 수 = Mini Batch의 수

### 1.2. 데이터 로드

데이터셋(Dataset), 데이터로더(DataLoader)가 제공된다

데이터로더: 데이터셋의 학습 방법을 결정

    batch_size = 2; 미니 배치의 크기는?
    shuffle = True; 미니 배치 data를 섞어 진행할건가?

> 미니 배치 학습

> 데이터 셔플(shuffle)

> 병렬처리

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [3]:
''' 사용법 예시 '''

# 데이터 예시
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# TensorDataset으로 저장하기
dataset = TensorDataset(x_train, y_train)

# DataLoader 정의
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)
  # Batch Size는 2의 지수승으로 정의한다: CPU, GPU의 용량을 최적으로 사용할 수 있음!

In [4]:
# 선형 다중 회귀모델로 적용 연습

model = nn.Linear(3,1) # 3변수->1변수
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-6)

nb_epochs = 100
for epoch in range(nb_epochs+1):
  for batch_idx, samples in enumerate(dataloader):
    # Batch Size: 2로 설정, x_train의 data는 5개
    # -> 5개중 (2,2,1)로 Batch를 만들어 학습을 진행한다!
    
    #print(batch_idx)
    #print(samples)
    
    # 미니 bacth data를 train 값으로 설정
    x_train, y_train = samples

    # H(x)
    prediction = model(x_train)

    # Cost
    cost = F.mse_loss(prediction, y_train)

    # Opti
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()


  
    '''print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))'''
  if epoch%10 == 0:
    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/100 Batch 3/3 Cost: 22393.015625
Epoch   10/100 Batch 3/3 Cost: 1316.724976
Epoch   20/100 Batch 3/3 Cost: 100.580063
Epoch   30/100 Batch 3/3 Cost: 11.861453
Epoch   40/100 Batch 3/3 Cost: 2.028981
Epoch   50/100 Batch 3/3 Cost: 0.550933
Epoch   60/100 Batch 3/3 Cost: 0.386782
Epoch   70/100 Batch 3/3 Cost: 0.569894
Epoch   80/100 Batch 3/3 Cost: 0.101110
Epoch   90/100 Batch 3/3 Cost: 0.389862
Epoch  100/100 Batch 3/3 Cost: 0.113172


#2. 커스텀 데이터셋(Custom Dataset)
https://wikidocs.net/57165

torch.utils.data.Dataset을 상속받는 Class를 만들어 커스터마이징 하기도 한다!

In [5]:
''' 기본 틀'''

class CustomDataset(torch.utils.data.Dataset):
  # 생성자; 데이터셋의 전처리 부분
  def __init__(self):
    return None

  # 데이터셋의 길이: 총 샘플의 수를 적어주는 부분
  def __len__(self):
    return None
  
  # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
  def __getitem__(self, idx):
    return None


In [6]:
''' Custom Dataset으로 선형 회귀 구현하기 '''

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Custom Dataset Class 구현
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73,80,75],
                   [93,88,93],
                   [89,91,90],
                   [96,98,100],
                   [73,66,70]]
    self.y_data = [[152],[185],[180],[196],[142]]

  # 총 데이터 개수 리턴
  def __len__(self):
    return len(self.x_data)
  
  # index(idx)값을 받아, 해당하는 Data를 추출하여 보여준다
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x,y


## dataset 정의, Dataloader 설정 (mini batch)
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)


## model 정의
model = nn.Linear(3,1) # 3변수 -> 1변수

## Optimizer 정의 : SGD
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

## 학습
nb_epochs = 50
for epoch in range(nb_epochs+1):
  # 미니배치 실행
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

  #epoch마다 출력
  print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
      epoch, nb_epochs, batch_idx+1, len(dataloader),
      cost.item()
      ))

Epoch    0/50 Batch 3/3 Cost: 9288.707031
Epoch    1/50 Batch 3/3 Cost: 181.724686
Epoch    2/50 Batch 3/3 Cost: 2.891650
Epoch    3/50 Batch 3/3 Cost: 0.308966
Epoch    4/50 Batch 3/3 Cost: 0.566995
Epoch    5/50 Batch 3/3 Cost: 0.825387
Epoch    6/50 Batch 3/3 Cost: 0.130062
Epoch    7/50 Batch 3/3 Cost: 0.041297
Epoch    8/50 Batch 3/3 Cost: 0.652541
Epoch    9/50 Batch 3/3 Cost: 0.536218
Epoch   10/50 Batch 3/3 Cost: 0.619493
Epoch   11/50 Batch 3/3 Cost: 0.484776
Epoch   12/50 Batch 3/3 Cost: 0.643378
Epoch   13/50 Batch 3/3 Cost: 0.705491
Epoch   14/50 Batch 3/3 Cost: 0.569226
Epoch   15/50 Batch 3/3 Cost: 0.510696
Epoch   16/50 Batch 3/3 Cost: 0.536442
Epoch   17/50 Batch 3/3 Cost: 0.150769
Epoch   18/50 Batch 3/3 Cost: 0.705773
Epoch   19/50 Batch 3/3 Cost: 0.325396
Epoch   20/50 Batch 3/3 Cost: 0.682358
Epoch   21/50 Batch 3/3 Cost: 0.612881
Epoch   22/50 Batch 3/3 Cost: 0.143489
Epoch   23/50 Batch 3/3 Cost: 0.191513
Epoch   24/50 Batch 3/3 Cost: 0.474609
Epoch   25/50 Batch 

In [7]:
# 결과 확인

new_var = torch.FloatTensor([[73,80,75]])
pred_y = model(new_var)
print("훈련 후 입력이 73,80,75일 때의 예측값: ", float(pred_y))

훈련 후 입력이 73,80,75일 때의 예측값:  151.52117919921875


#3. 토치비전(Torchvision)

https://wikidocs.net/60324

하기 라이브러리들을 포함하고 있는 패키지이다.
* 유명한 dataset들
* 구현되어있는 유명한 모델들
* 일반적인 이미지 전처리 도구들
> info: https://pytorch.org/docs/stable/torchvision/index.html


In [8]:
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

# 4. 현재 환경에서 GPU 연산 가능하게 하기

https://wikidocs.net/60324

In [9]:
import torch

USE_CUDA = torch.cuda.is_available() # GPU를 사용 가능하면 True, 아니면 False 리턴
device = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다: ", device)

다음 기기로 학습합니다:  cuda
